# Universal RO System Simulation Engine

This notebook serves as a universal simulation engine for RO systems, leveraging modular utility functions.
It supports multiple property packages (NaCl, MCAS) and simulation strategies.

In [ ]:
# Parameters cell - will be replaced by papermill
project_root = "/path/to/project"  # Will be replaced by papermill
configuration = {}  # RO configuration from optimization
feed_salinity_ppm = 5000
feed_temperature_c = 25.0
membrane_type = "brackish"
membrane_properties = None
optimize_pumps = False
feed_ion_composition = None  # JSON string or dict for MCAS
property_package = "auto"  # "nacl", "mcas", or "auto"
initialization_strategy = "sequential"  # "sequential", "block_triangular", "custom_guess", "relaxation"

## Setup and Imports

In [ ]:
# System imports
import sys
import os
import json
import warnings
from pathlib import Path

# Add project root to path
sys.path.insert(0, project_root)

# Import all utility modules
from utils.mcas_builder import (
    build_mcas_property_configuration,
    check_electroneutrality,
    get_total_dissolved_solids,
    calculate_ionic_strength
)
from utils.ro_model_builder import build_ro_model_mcas
from utils.ro_initialization import (
    initialize_multistage_ro_elegant,
    get_stream_tds_ppm
)
from utils.ro_solver import (
    initialize_and_solve_mcas,
    initialize_model_advanced
)
from utils.ro_results_extractor import extract_results_mcas
from utils.scaling_prediction import (
    predict_scaling_potential,
    get_antiscalant_recommendation
)
from utils.simulate_ro import (
    run_ro_simulation,
    calculate_lcow,
    estimate_capital_cost
)
from utils.validation import validate_ion_composition
from utils.response_formatter import format_simulation_response
from utils.config import get_config
from utils.logging_config import get_configured_logger

# Configure logging
logger = get_configured_logger(__name__)
warnings.filterwarnings('ignore')

# Results storage
results = {}

## Determine Property Package

In [ ]:
# Determine which property package to use
if property_package == "auto":
    # Auto-detect based on feed composition
    if feed_ion_composition:
        property_package = "mcas"
        logger.info("Auto-selected MCAS property package based on ion composition")
    else:
        property_package = "nacl"
        logger.info("Auto-selected NaCl property package (no ion composition provided)")

print(f"Using property package: {property_package}")
print(f"Initialization strategy: {initialization_strategy}")

## Validate and Prepare Inputs

In [ ]:
# Validate configuration
if not isinstance(configuration, dict) or "stages" not in configuration:
    raise ValueError("Invalid configuration provided")

# Parse ion composition if provided
parsed_ion_composition = None
if feed_ion_composition:
    if isinstance(feed_ion_composition, str):
        parsed_ion_composition = json.loads(feed_ion_composition)
    else:
        parsed_ion_composition = feed_ion_composition
    
    # Validate ion composition
    validate_ion_composition(parsed_ion_composition)

# Add feed_flow_m3h to configuration if missing
if "feed_flow_m3h" not in configuration:
    if "recycle_info" in configuration and "effective_feed_flow_m3h" in configuration["recycle_info"]:
        configuration["feed_flow_m3h"] = configuration["recycle_info"]["effective_feed_flow_m3h"]
    elif "stages" in configuration and configuration["stages"]:
        first_stage = configuration["stages"][0]
        if "feed_flow_m3h" in first_stage:
            configuration["feed_flow_m3h"] = first_stage["feed_flow_m3h"]
        else:
            configuration["feed_flow_m3h"] = 100.0
    else:
        configuration["feed_flow_m3h"] = 100.0

print(f"Configuration: {configuration['array_notation']}")
print(f"Feed flow: {configuration['feed_flow_m3h']} m³/h")
print(f"Feed salinity: {feed_salinity_ppm} ppm")
print(f"Feed temperature: {feed_temperature_c}°C")

## Run Simulation Using Utility Functions

In [ ]:
# Run the simulation using the modular utility function
try:
    logger.info(f"Starting {property_package.upper()} simulation...")
    
    # Use the centralized simulation function
    sim_results = run_ro_simulation(
        configuration=configuration,
        feed_salinity_ppm=feed_salinity_ppm,
        feed_ion_composition=parsed_ion_composition,
        feed_temperature_c=feed_temperature_c,
        membrane_type=membrane_type,
        membrane_properties=membrane_properties,
        optimize_pumps=optimize_pumps,
        use_nacl_equivalent=(property_package == "nacl"),
        initialization_strategy=initialization_strategy
    )
    
    # Format results for output
    results = format_simulation_response(sim_results)
    results["property_package"] = property_package
    results["initialization_strategy"] = initialization_strategy
    
    logger.info("Simulation completed successfully")
    
except Exception as e:
    logger.error(f"Simulation failed: {str(e)}", exc_info=True)
    results = {
        "status": "error",
        "message": f"Simulation failed: {str(e)}",
        "error_type": type(e).__name__,
        "property_package": property_package,
        "initialization_strategy": initialization_strategy
    }

## Display Results Summary

In [ ]:
# Display results summary
print("\n" + "="*60)
print(f"SIMULATION RESULTS - {property_package.upper()} MODEL")
print("="*60)

if results.get("status") == "success":
    print(f"\nOverall Performance:")
    print(f"  Total Recovery: {results['performance']['system_recovery']:.1%}")
    print(f"  Specific Energy: {results['economics']['specific_energy_kwh_m3']:.2f} kWh/m³")
    print(f"  Total Power: {results['economics']['total_power_kw']:.1f} kW")
    print(f"  LCOW: ${results['economics'].get('lcow_usd_m3', 0):.2f}/m³")
    
    if "ion_analysis" in results and results["ion_analysis"]:
        print(f"\nIon Rejection:")
        for ion, rejection in results['ion_analysis'].get('overall_rejection', {}).items():
            print(f"  {ion:8s}: {rejection:.1%}")
    
    print(f"\nStage Summary:")
    for i, stage in enumerate(results['stage_results'], 1):
        print(f"  Stage {i}: {stage['feed_pressure_bar']:.1f} bar, "
              f"Recovery {stage['water_recovery']:.1%}, "
              f"Permeate {stage['permeate_tds_ppm']:.0f} ppm")
        
        # Show scaling warnings if any
        if stage.get('scaling_indices'):
            high_risk = [k for k, v in stage['scaling_indices'].items() if v > 0]
            if high_risk:
                print(f"    ⚠️  Scaling risk: {', '.join(high_risk)}")
else:
    print(f"\n❌ Simulation failed: {results.get('message', 'Unknown error')}")

## Export Full Results

In [ ]:
# Results cell - tagged for papermill to extract
results

## Optional: Save Results to File

In [ ]:
# Optionally save results to JSON file
if results.get("status") == "success":
    import json
    from datetime import datetime
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = Path(project_root) / "results" / f"simulation_results_{timestamp}.json"
    
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2, default=str)
    
    print(f"\nResults saved to: {output_file}")